# Évaluation du risque de réidentification dans les bases de données de santé
## Projet M2 MIAS - Février 2026

---

## 1. Contexte et Objectifs
Ce projet vise à imaginer et tester des indicateurs quantitatifs du risque de réidentification d'individus dans une base de données, conformément aux critères du G29 (CNIL) : **Inférence**, **Corrélation**, et **Individualisation**.

Nous travaillerons sur des données synthétiques simulant des séjours hospitaliers et analyserons l'impact de deux techniques de protection :
1. **Mélange de valeurs (Permutation)** : Échange de valeurs entre individus pour une même variable.
2. **Échantillonnage avec remise** : Création d'une base issue d'un tirage aléatoire, modifiant la population initiale.

Les fichiers analysés sont :
- `connaissances_externes.txt` : La base de référence (population source).
- `out_direct_XX.txt` : Base avec XX% de mélange (conservation de la population).
- `out_sample_XX.txt` : Base échantillonnée avec XX% de mélange.

## 2. Configuration et Simulation des Données

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import entropy
import warnings
warnings.filterwarnings('ignore')

# Configuration graphique
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
np.random.seed(2026)

def simuler_base_donnees(n=1000):
    """Génère une base de données synthétique réaliste"""
    df = pd.DataFrame()
    df['id_sejour'] = range(1, n+1)
    
    # Données démographiques
    ages = np.random.randint(18, 95, n)
    df['age'] = ages
    df['age5'] = (ages // 5) * 5
    df['age10'] = (ages // 10) * 10
    df['sexe'] = np.random.choice(['M', 'F'], n, p=[0.48, 0.52])
    
    # Données temporelles
    dates_entree = pd.date_range('2023-01-01', periods=n, freq='H')
    durees = np.random.exponential(5, n).astype(int) + 1
    dates_sortie = dates_entree + pd.to_timedelta(durees, unit='D')
    
    df['entree_date_ymd'] = dates_entree.strftime('%Y-%m-%d')
    df['entree_date_ym'] = dates_entree.strftime('%Y-%m')
    df['entree_date_y'] = dates_entree.strftime('%Y')
    
    # Données médicales
    specialites = ['Cardio', 'Pneumo', 'Gastro', 'Neuro', 'Trauma', 'Onco']
    df['specialite'] = np.random.choice(specialites, n)
    df['chirurgie'] = np.random.choice([0, 1], n, p=[0.7, 0.3])
    df['diabete'] = np.random.choice([0, 1], n, p=[0.85, 0.15])
    
    # Mélange initial des colonnes
    cols = list(df.columns)
    cols.remove('id_sejour')
    np.random.shuffle(cols)
    return df[['id_sejour'] + cols]

# Création de la base de référence
base_reference = simuler_base_donnees(1000)
print(f"Base générée : {base_reference.shape}")
base_reference.head()

## 3. Génération des fichiers modifiés (Direct et Sample)

In [2]:
def melanger_colonnes(df, pourcentage):
    """Mélange aléatoire des valeurs dans chaque colonne"""
    df_res = df.copy()
    if pourcentage == 0: return df_res
    
    for col in df.columns:
        if col == 'id_sejour': continue
        n_mix = int(len(df) * pourcentage / 100)
        idx = np.random.choice(len(df), n_mix, replace=False)
        vals = df_res.loc[idx, col].values
        np.random.shuffle(vals)
        df_res.loc[idx, col] = vals
    return df_res

def echantillonner(df, pourcentage_melange):
    """Tirage avec remise puis mélange"""
    idx = np.random.choice(len(df), len(df), replace=True)
    df_sample = df.iloc[idx].reset_index(drop=True)
    # Conservation des IDs pour vérification (théorique)
    df_sample['id_sejour'] = df['id_sejour'].iloc[idx].values
    return melanger_colonnes(df_sample, pourcentage_melange)

fichiers = {'reference': base_reference}
niveaux = [0, 10, 20, 30, 40, 50]

for n in niveaux:
    fichiers[f'direct_{n}'] = melanger_colonnes(base_reference, n)
    fichiers[f'sample_{n}'] = echantillonner(base_reference, n)

## 4. Définition des Indicateurs de Risque

Nous implémentons trois métriques principales :
1. **Taux d'individualisation** : Pourcentage d'enregistrements uniques dans la base anonymisée qui correspondent à un individu unique dans la base source (réidentification réussie).
2. **Risque d'inférence** : Capacité à deviner une valeur sensible (ex: diabète) connaissant les autres attributs.
3. **Risque de corrélation (Entropie)** : Mesure de la perte d'information/structure.

In [3]:
class Indicateurs:
    @staticmethod
    def individualisation(df_base, df_anon, qi_vars):
        """Calcule le taux de réidentification correcte unique"""
        # Dans la base anon, combien sont uniques sur les quasi-identifiants ?
        groupes = df_anon.groupby(qi_vars).size()
        uniques_anon = groupes[groupes == 1].index
        
        # Filtrer la base anon pour ne garder que ces uniques
        df_unique = df_anon.set_index(qi_vars).loc[uniques_anon].reset_index()
        
        reussites = 0
        # Vérification avec la base source (attaque par jointure)
        # Note: Optimisé pour la démonstration
        for _, row in df_unique.iterrows():
            # Trouver les correspondances dans la source
            mask = np.ones(len(df_base), dtype=bool)
            for v in qi_vars:
                mask &= (df_base[v] == row[v])
            match = df_base[mask]
            
            if len(match) == 1:
                # Succès si l'ID correspond (l'attaquant ne connait pas l'ID, mais nous vérifions le risque)
                if match.iloc[0]['id_sejour'] == row['id_sejour']:
                    reussites += 1
        
        return (reussites / len(df_anon)) * 100

    @staticmethod
    def inference(df_base, df_anon, target_var):
        """Mesure la précision de l'inférence simple (valeur conservée)"""
        # Si les données sont mélangées, la valeur observée peut être fausse
        # On mesure ici simplement si la valeur dans anon correspond à la valeur réelle
        # pour les mêmes lignes (simplification)
        if len(df_base) != len(df_anon): return 0 # Non applicable directement sur sample sans alignement
        correct = (df_base[target_var].values == df_anon[target_var].values).sum()
        return correct / len(df_base) * 100

    @staticmethod
    def unicite(df, vars):
        return (df.groupby(vars).size() == 1).sum() / len(df) * 100

## 5. Analyse par Scénarios

Définition de scénarios de connaissances de l'attaquant, allant de faible à fort.

In [4]:
scenarios = [
    {'nom': 'S1: Faible', 'vars': ['age10', 'sexe', 'entree_date_y']},
    {'nom': 'S2: Moyen', 'vars': ['age5', 'sexe', 'entree_date_ym', 'specialite']},
    {'nom': 'S3: Fort', 'vars': ['age', 'sexe', 'entree_date_ymd', 'specialite', 'chirurgie']}
]

resultats = []

for nom_f, df in fichiers.items():
    if nom_f == 'reference': continue
    type_f = 'Direct' if 'direct' in nom_f else 'Sample'
    niveau = int(nom_f.split('_')[1])
    
    for scen in scenarios:
        risk_ind = Indicateurs.individualisation(base_reference, df, scen['vars'])
        unicite_val = Indicateurs.unicite(df, scen['vars'])
        
        res = {
            'Fichier': type_f,
            'Mélange (%)': niveau,
            'Scénario': scen['nom'],
            'Risque Individualisation (%)': risk_ind,
            'Unicité (%)': unicite_val
        }
        resultats.append(res)

df_res = pd.DataFrame(resultats)
df_res.head()

## 6. Visualisation des Résultats

### 6.1 Impact du Mélange sur le Risque

In [5]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_res[df_res['Fichier'] == 'Direct'], 
             x='Mélange (%)', y='Risque Individualisation (%)', 
             hue='Scénario', marker='o')
plt.title("Évolution du Risque d'Individualisation (Fichiers Directs)")
plt.ylim(0, 105)
plt.show()

### 6.2 Comparaison Direct vs Sample

In [6]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_res, x='Mélange (%)', y='Risque Individualisation (%)', 
             hue='Fichier', style='Scénario', markers=True, dashes=False)
plt.title("Comparaison de l'efficacité : Permutation (Direct) vs Échantillonnage (Sample)")
plt.ylabel("Taux de Réidentification réussie (%)")
plt.grid(True, alpha=0.3)
plt.show()

## 7. Discussion et Conclusion

### Analyse des résultats

1.  **Impact des connaissances (Scénarios) :**
    * Plus l'attaquant possède d'informations précises (Scénario 3 vs Scénario 1), plus le risque de réidentification est élevé (proche de 100% sans protection). L'unicité des individus augmente drastiquement avec le nombre de variables.

2.  **Effet du Mélange (Permutation) :**
    * Le risque diminue linéairement avec le taux de mélange. En cassant les corrélations entre les quasi-identifiants (ex: âge, code postal) et les identifiants sensibles, on réduit la capacité à isoler un individu unique avec certitude.

3.  **Supériorité de l'Échantillonnage (`Sample` vs `Direct`) :**
    * Comme le montre le graphique comparatif, la méthode **Sample (Échantillonnage avec remise)** offre systématiquement une meilleure protection (risque plus faible) à niveau de mélange égal.
    * **Pourquoi ?** L'échantillonnage introduit une incertitude fondamentale : un individu présent dans la base anonymisée n'est pas forcément unique dans la population (duplication), et un individu de la population cible peut ne pas être dans la base (absence).
    * Dans le fichier `Direct`, la population est conservée (bijection) : si je trouve un profil unique qui correspond à ma cible, c'est forcément elle (sauf si les données sont mélangées). Dans le fichier `Sample`, même une correspondance unique n'est pas une preuve formelle d'appartenance.

### Conclusion
Pour minimiser le risque de réidentification, il est recommandé de combiner la **généralisation** des données (réduire la précision, cf. Scénario 1) et l'**échantillonnage**, plutôt que de simplement mélanger les données d'une population exhaustive.